In [1]:
import os
import pathlib
from dotenv import load_dotenv
import boto3
import sagemaker

# Step 1: Normalize working dir (repo root)
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
print("Working dir:", pathlib.Path().resolve())

# Step 2: Load .env if available (local dev), else fall back to Studio role
creds = {}
region = None

if os.path.exists(".env"):
    load_dotenv(dotenv_path=".env", override=True)
    region = os.getenv("AWS_REGION")
    creds = {
        "aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID"),
        "aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY")
    }
    print("✅ Loaded AWS creds from .env")
else:
    region = sagemaker.Session().boto_region_name
    print("✅ Using SageMaker Studio execution role")

print("Region:", region)

# Step 3: Create SageMaker runtime + client
runtime = boto3.client("sagemaker-runtime", region_name=region, **creds)
sm_client = boto3.client("sagemaker", region_name=region, **creds)

print("✅ SageMaker runtime + client ready")



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Working dir: /home/sagemaker-user/SageNotebook
✅ Using SageMaker Studio execution role
Region: us-east-1
✅ SageMaker runtime + client ready


In [2]:
hub = {
    "HF_MODEL_ID": "distilbert-base-uncased-finetuned-sst-2-english",
    "HF_TASK": "text-classification"
}
print("✅ Hugging Face config ready")


✅ Hugging Face config ready


In [3]:
from sagemaker.huggingface import HuggingFaceModel

huggingface_model = HuggingFaceModel(
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",
    env=hub,
    role=sagemaker.get_execution_role()
)
print("✅ HuggingFace model object created")


✅ HuggingFace model object created


In [5]:
import boto3
sm_client = boto3.client("sagemaker")

endpoint_name = "huggingface-sentiment"

try:
    sm_client.delete_endpoint(EndpointName=endpoint_name)
    print(f"✅ Deleted endpoint: {endpoint_name}")
except sm_client.exceptions.ClientError:
    print("⚠️ Endpoint not found, skipping")

try:
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"✅ Deleted endpoint config: {endpoint_name}")
except sm_client.exceptions.ClientError:
    print("⚠️ Endpoint config not found, skipping")


✅ Deleted endpoint: huggingface-sentiment
✅ Deleted endpoint config: huggingface-sentiment


In [6]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",   
    endpoint_name="huggingface-sentiment"
)
print("✅ Endpoint deployed: huggingface-sentiment")


------!✅ Endpoint deployed: huggingface-sentiment


In [7]:
result = predictor.predict({"inputs": "This SageMaker setup is fantastic!"})
print("Prediction:", result)


Prediction: [{'label': 'POSITIVE', 'score': 0.9998700618743896}]


In [8]:
predictor.delete_endpoint()
print("✅ Endpoint deleted to save costs")


✅ Endpoint deleted to save costs


In [ ]:
```python
       import os
       import pandas as pd
       from sklearn.model_selection import train_test_split
       from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
       import torch

       def load_data(data_dir):
           df = pd.read_csv(os.path.join(data_dir, 'banking_complaints.csv'))
           df = df.merge(pd.read_csv(os.path.join(data_dir, 'dept_product.csv')).rename(columns={'Product': 'Banking Product', 'Department': 'department'}), on='Banking Product', how='left')
           syn_map = {'Credit reporting': 'Credit Reports', 'Credit card': 'Credit Cards', 'Prepaid card': 'Credit Cards', 'Consumer Loan': 'Loans', 'Payday loan': 'Loans', 'Bank account or service': 'CASA', 'Money transfers': 'Remittance'}
           df['department'] = df['department'].fillna(df['Banking Product'].map(syn_map))
           df = df.dropna(subset=['department']).reset_index(drop=True)
           return df

       if __name__ == '__main__':
           data_dir = '/opt/ml/input/data/training'
           df = load_data(data_dir)
           tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
           labels = {dept: i for i, dept in enumerate(df['department'].unique())}
           df['label'] = df['department'].map(labels)

           train_texts, val_texts, train_labels, val_labels = train_test_split(df['Complaint Description'], df['label'], test_size=0.2, random_state=42)

           train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
           val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)

           class ComplaintDataset(torch.utils.data.Dataset):
               def __init__(self, encodings, labels):
                   self.encodings = encodings
                   self.labels = labels
               def __getitem__(self, idx):
                   item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                   item['labels'] = torch.tensor(self.labels[idx])
                   return item
               def __len__(self):
                   return len(self.labels)

           train_dataset = ComplaintDataset(train_encodings, train_labels.tolist())
           val_dataset = ComplaintDataset(val_encodings, val_labels.tolist())

           model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(labels))
           training_args = TrainingArguments(
               output_dir='/opt/ml/model',
               num_train_epochs=3,
               per_device_train_batch_size=16,
               per_device_eval_batch_size=16,
               warmup_steps=500,
               weight_decay=0.01,
               logging_dir='/opt/ml/logs',
               logging_steps=10,
               evaluation_strategy='epoch',
               save_strategy='epoch',
               load_best_model_at_end=True
           )

           trainer = Trainer(
               model=model,
               args=training_args,
               train_dataset=train_dataset,
               eval_dataset=val_dataset
           )

           trainer.train()
           trainer.save_model('/opt/ml/model')
       ```